# Resultados GloVe
En este notebook nos encargamos de entrenar los modelos y mostrar resultados para cada uno de ellos. Mostraremos precisión, sensibilidad y F1 para cada categoría así como la media que estos ofrecen. Como técnica de conversión a formato numérico utilizaremos GloVe.

### Activamos Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importamos librerías necesarias



In [ ]:
import pandas as pd
import numpy as np
import random
import math
import re
import warnings

from sklearn.utils import shuffle

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import normalize

import ipywidgets as widgets

import nltk
from nltk.corpus import wordnet
from joblib import dump, load

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

!pip install spacy
!python -m spacy download en_core_web_lg

import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 827.9MB 1.1MB/s 


### Definimos función que nos dice si una palabra es un adjetivo, un nombre, un verbo o un advervio.

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

### Definimos función que recibe una cadena de texto y devuelve el texto procesado.

In [ ]:
def preprocess(sentence):
    # Convertimos a minúsculas
    new_text = sentence.lower()
    
    # Eliminamos puntuación
    new_text = re.sub(r'[^\w\s]', '', new_text)

    # Dividimos en tokens
    tokens = nltk.tokenize.TreebankWordTokenizer().tokenize(new_text)

    # Eliminamos stopwords
    tokens = [word for word in tokens if not word in nltk.corpus.stopwords.words('english')]
    new_text = ' '.join(word for word in tokens)

    # Stemming
    # stemmer = nltk.stem.PorterStemmer()
    # new_text = ' '.join([stemmer.stem(w) for w in tokens])
    
    # lemma
    lemmatizer = nltk.stem.WordNetLemmatizer()
    new_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens])

    # Reemplazamos números por #s
    if bool(re.search(r'\d', new_text)):
        new_text = re.sub('[0-9]{5,}', '#####', new_text)
        new_text = re.sub('[0-9]{4}', '####', new_text)
        new_text = re.sub('[0-9]{3}', '###', new_text)
        new_text = re.sub('[0-9]{2}', '##', new_text)
        # Cuando existe un solo número lo eliminamos
        new_text = re.sub('[0-9]{1}', '', new_text)

    return new_text

###Definimos función que calcula la media armónica de 2 números

In [ ]:
def calculate_f1_score(precision, recall):
    return 2*((precision * recall) / (precision + recall))

### Seleccionamos el modelo que queremos utilizar. Por defecto se utilizará k-NN con k=1.

In [ ]:
model_list = [
    ('K-NN con k=1', 0), 
    ('SVM núcleo lineal', 1), 
    ('Clasificador bayesiano ingenuo', 2), 
    ('Bosque aleatorio', 3)
]
model_picker = widgets.Dropdown(options=model_list)
print("Selecciona un modelo: ")
model_picker

### Cargamos el modelo GloVe para poder realizar las conversiones de cadena de texto a vector de números

In [ ]:
nlp = spacy.load('en_core_web_lg')

### Cargamos y preprocesamos la base de datos

In [ ]:
RUTA_DB = "./drive/My Drive/xlsx/db.xlsx"
db = pd.read_excel(RUTA_DB)
categories = [
        "access control", "audit", "availability", "legal", "look and feel",
        "maintainability", "operational", "privacy", "recoverability", "capacity and performance",
        "reliability", "security", "usability", "other nonfunctional", "functional", "not applicable"
    ]

# Creamos diccionario con las traducciones para cada categoría
cat_translations =[
    "Control de acceso", "Auditoría", "Disponibilidad", "Legal", "Diseño",
    "Mantenibilidad", "Operacional", "Privacidad", "Recuperabilidad", "Rendimiento",
    "Fiabilidad", "Seguridad", "Usabilidad", "Otros no funcionales", "Funcional", "No aplicable"    
]

db['sentences'] = db['sentences'].apply(preprocess)
db = db.dropna()
db = db.sample(frac=1, random_state = 5).reset_index(drop=True)

### Dividimos los datos en datos de entrada y clasificación

In [ ]:
x = normalize([nlp(s).vector for s in db['sentences'].values])
y = db.drop(labels=['sentences'], axis=1)

### Entrenamos modelo, validamos y mostramos resultados de precisión, sensibilidad y F1

In [ ]:
# Suprimimos posibles alertas
warnings.filterwarnings("ignore") 

# Definimos lista con los posibles modelos
models = [KNeighborsClassifier(n_neighbors=1), svm.LinearSVC(), naive_bayes.BernoulliNB(), RandomForestClassifier(n_jobs=-1)]
# Escogemos el modelo que vamos a utilizar
model = models[model_picker.value]

# Definimos variables necesarias para el almacenamiento de los resultados
n_splits = 10
mean_precision = [[0]*16]*n_splits
mean_recall = [[0]*16]*n_splits
mean_f1score = [[0]*16]*n_splits
i_test = 0 
f_test = math.floor(len(db)/n_splits)

# Iteramos n_splits veces (Validación cruzada con n_splits = 10)
for i_split in range(n_splits):
    print("\nIteración", i_split+1, end='')
    # Definimos variables de precisión, sensibilidad y F1 para esta iteración 
    precision = [0]*len(categories)
    recall = [0]*len(categories)
    f1score = [0]*len(categories)

    # Dividimos en vectores de características de entrenamiento y validación,
    # y clasificaciones para entrenamiento y validaciób
    x_train = list(x)[:i_test] + list(x)[f_test:]
    y_train = y.drop(range(i_test, f_test))
    x_test = list(x)[i_test:f_test]
    y_test = y[i_test:f_test]

    # Para cada categoría
    for i, category in enumerate(categories):
        print('.', end='')
        # Entrenamos el modelo
        model.fit(x_train, y_train[category])
        # Realizamos predicción para conjunto de prueba
        prediction = model.predict(x_test)
        # Obtenemos resultados
        precision[i] = precision_score(list(y_test[category].values), prediction)
        recall[i] = recall_score(list(y_test[category].values), prediction)

    i_test = f_test
    f_test = i_test + math.floor(len(db)/n_splits)

    mean_precision[i_split] = precision
    mean_recall[i_split] = recall
    mean_f1score[i_split] = f1score

# Mostramos resultados agrupados en un dataframe de pandas
p = list(np.average(mean_precision, axis=0))
rounded_p = [round(elem, 2) for elem in p]
r = list(np.average(mean_recall, axis=0))
rounded_r = [round(elem, 2) for elem in r]
rounded_f1 = [
    round(calculate_f1_score(rounded_p[i], rounded_r[i]), 2) for i, elem in enumerate(rounded_p)
]
data = {
    " ": cat_translations + ['MICRO AVG'],
    "Precisión": rounded_p + [np.average(rounded_p)],
    "Sensibilidad": rounded_r + [np.average(rounded_r)],
    "F1": rounded_f1 + [calculate_f1_score(np.average(rounded_p), np.average(rounded_r))]
}
print("\nLos resultados para el modelo seleccionado son:")
pd.options.display.float_format = "{:,.2f}".format
pd.DataFrame(data).set_index(' ')


### Exportamos los modelos
Los modelos que exportaremos coincidiran con los entrenados en la primera
iteración de la validación cruzada. De esta manera podremos utilizar estos modelos en otro lugar para poder clasificar nuevos elementos.

In [ ]:
i_test = 0
f_test = math.floor(len(db)/n_splits)

x_train = list(x)[:i_test] + list(x)[f_test:]
y_train = y.drop(range(i_test, f_test))
x_test = list(x)[i_test:f_test]
y_test = y[i_test:f_test]

# Para cada categoría
for category in categories[:-1]:
    model = models[model_picker.value]
    model.fit(x_train, y_train[category])
    dump(model, './drive/My Drive/nfr-extraction/models-glove/' + category.replace(" ", "") + '.joblib')
